In [1]:
# Bibliotecas PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F

# Bibliotecas Torchvision
from torchvision import datasets, models, transforms
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights

# Outras
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import os
import copy
import random
import zipfile
from google.colab import drive

# Métricas Sklearn para avaliação
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# Passo 1: Acessar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Montar o dataset, separando em treino, validação e teste
train_dir ='/content/drive/MyDrive/Priori-RX/dataset_NIH/train'
val_dir = '/content/drive/MyDrive/Priori-RX/dataset_NIH/validation'
test_dir = '/content/drive/MyDrive/Priori-RX/dataset_NIH/test'

In [4]:
# Data argumentation nos conjuntos de treino, validação e treino
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
# Carregar dataset com ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, transform=data_transforms['val']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}

In [6]:
# Evitar multiprocessamento
batch_size = 32
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=0),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True, num_workers=0),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False, num_workers=0)
}

In [7]:
# Obter classes encontradas e quantidade total de imagens por conjunto
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes
print('Classes encontradas', class_names)
print('Total de imagens encontradas por conjunto: ', dataset_sizes)

Classes encontradas ['Doente', 'Normal']
Total de imagens encontradas por conjunto:  {'train': 17513, 'val': 7486, 'test': 40}


In [8]:
# Carregando o modelo pré-treinado EfficientNet-B7 - possui 7 blocos principais em model.features
model = efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1)

# Congelando camadas
for param in model.parameters():
    param.requires_grad = False
for param in model.features[6:].parameters():  # deixando somente os dois últimos blocos descongelados
    param.requires_grad = True

# Modificando as últimas camadas e incrementando um dropout
num_ftrs = model.classifier[1].in_features  # Obtendo os recursos de entrada da camada final
model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Dropout(0.5),      # drouptou para regularização
    nn.Linear(num_ftrs, 512),  # camada intermidária
    nn.ReLU(),                 # Activation function
    nn.Dropout(0.3),      # outro dropout
    nn.Linear(512, 2)     # camada final ajustada para classificação binária
)

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-c5b4e57e.pth
100%|██████████| 255M/255M [00:02<00:00, 121MB/s]


In [9]:
# Função de perda com suavização de rótulo
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Definir otimizador com maior redução de peso
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=5e-4)

# ajuste da taxa de aprendizado com base na função cosseno
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Definindo o Early stopping e checkpoints
patience = 5
epochs_without_improvement = 0
best_val_loss = float('inf')
best_model_wts = copy.deepcopy(model.state_dict())

In [10]:
import torch

# Definir o device (GPU se disponível, caso contrário, CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mover o modelo para o dispositivo
model = model.to(device)

In [11]:
# Loop de treinamento com acumulação de gradiente
num_epochs = 100
accumulation_steps = 4  # acumula o gradiente a cada 4 batchs
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []  # Armazenar previsões

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Fase de treino
    for i, (images, labels) in enumerate(tqdm(dataloaders['train'], desc=f"Epoch {epoch+1}/{num_epochs}")):
        images, labels = images.to(device), labels.to(device)

        # Cálculo Backward e perda
        outputs = model(images)
        loss = criterion(outputs, labels) / accumulation_steps  # Normalize loss by accumulation steps

        # Backward
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()  # Atualizar os pesos depois do acumulo de gradiente
            optimizer.zero_grad()

        running_loss += loss.item() * labels.size(0)

         # Cálculo da acurácia
        _, preds = torch.max(outputs, 1)  # Obter classe prevista
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_loss / dataset_sizes['train']
    train_acc = correct_train / total_train  # Acurácia do treino
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Fase de validação
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * labels.size(0)

            # Cálculo da acurácia
            _, preds = torch.max(outputs, 1)  # Obter classe prevista
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    val_loss = val_running_loss / dataset_sizes['val']
    val_acc = correct_val / total_val  # Acurácia da validação
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    # Checando o Early stopping - baseado no loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement == patience:
            print("Early stopping ativado.")
            break

    # Ajuste da taxa de aprendizado
    scheduler.step()

    # Print dos resultados por época
    print(f"Epoch {epoch+1}/{num_epochs} - Train Acc: {train_acc:.4f} - Train Loss: {train_loss:.4f} x Val Acc: {val_acc:.4f} - Val Loss: {val_loss:.4f}, ")


# Carregar modelo com melhores pesos
model.load_state_dict(best_model_wts)

Epoch 1/100:   0%|          | 2/548 [01:06<5:01:02, 33.08s/it]


KeyboardInterrupt: 

In [ ]:
# Plotando gráficos com resultados
plt.plot(train_losses, label='Training Loss', color='blue', linestyle='-', marker='o', markersize=4)
plt.plot(val_losses, label='Validation Loss', color='red', linestyle='--', marker='x', markersize=4)
plt.plot(train_accuracies, label='Training Accuracy', color='green', linestyle='-', marker='o', markersize=4)
plt.plot(val_accuracies, label='Validation Accuracy', color='yellow', linestyle='--', marker='x', markersize=4)

plt.title('Training and Validation over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Parametros', fontsize=14)

plt.legend()

plt.grid(True)
plt.show()

In [ ]:
# função para carregar e processar imagem
def load_and_preprocess_image(image_path, transform):
    """
    Carrega uma imagem, converte para RGB, aplica transformações e transforma num tensor para servir de entrada pro modelo treinado.

    Argumentos da função:
        image_path (str): caminho da imagem a ser carregada.
        transform (torchvision.transforms.Compose): transformações aplicadas à imagem.

    Retorno:
        tuple: imagem original e tensor pré-processado pronto para entrar no modelo.
    """
    image = Image.open(image_path).convert("RGB")
    return image, transform(image).unsqueeze(0)  # Aplicar transformações e adicionar dimensão de lote

# Função de predição com carregamento, pré-processamento e predição
def predict_image(model, image_path, transform, device):
    """
    Carrega e processa a imagem, usando o modelo para prever a classe presente na imagem.

    Argumentos:
        model (torch.nn.Module): modelo de rede treinado .
        image_path (str): caminho da imagem a ser analisada.
        transform (torchvision.transforms.Compose): transformações que serão aplicadas a imagem.
        device (torch.device): dispositivo que vai rodar o modelo (CPU ou GPU).

    Retorno:
        tuple: imagem original e classe prevista (como uma matriz numpy).
    """
    original_image, image_tensor = load_and_preprocess_image(image_path, transform)
    model.eval()  # colocando o modelo no modo de avaliação
    with torch.no_grad():  # Desabilitar cálculo de gradiente para inferência
        image_tensor = image_tensor.to(device)  # muda tensor para o dispositivo indicado
        outputs = model(image_tensor)           # obter a saída do modelo
        probabilities = F.softmax(outputs, dim=1)  # aplicando softmax para obter a  classe prevista

    return original_image, probabilities.cpu().numpy().flatten()


# Função para visualizar as previsões
def visualize_predictions(image, probabilities, class_names, top_k=2):
    """
    Exibe a imagem original ao lado de um gráfico de barras das previsões.

    Argumentos:
        image (PIL.Image): imagem original
        probabilities (np.array): Matriz de probabilidades de classe geradas pelo modelo.
        class_names (list): Lista de nomes de classes correspondentes à saída do modelo.
        top_k (int): Número das principais previsões a serem exibidas (o padrão é 2 para classificação binária).
    """
    top_k_idx = probabilities.argsort()[-top_k:][::-1]         # Obtendo o indice de cada previsão
    top_classes = [class_names[i] for i in top_k_idx]          # Recuperar os nomes de classes
    top_probabilities = probabilities[top_k_idx]               # Obter probabilidades para as principais previsões

    # Plotando a imagem e suas previsões
    fig, axarr = plt.subplots(1, 2, figsize=(12, 6))
    axarr[0].imshow(image)
    axarr[0].axis("off")

    axarr[1].barh(top_classes, top_probabilities)
    axarr[1].set_xlabel("Probability")
    axarr[1].set_title("Top Class Predictions")
    axarr[1].invert_yaxis()

    plt.tight_layout()
    plt.show()


# Função para selecionar uma imagem aleatória da pasta de teste
def get_random_image_from_folder(folder_path):
    """
    Seleciona uma imagem aleatória do conjunto de teste

    Argumentos:
        folder_path (str): caminho da pasta "teste"

    Retorno:
        tuple: caminho para a imagem selecionada.
    """
    classes = os.listdir(folder_path)
    random_class = random.choice(classes)
    class_folder = os.path.join(folder_path, random_class)
    image_files = os.listdir(class_folder)
    random_image_file = random.choice(image_files)
    return os.path.join(class_folder, random_image_file), random_class

# Prevendo 10 imagens aleatórias
test_folder_path = test_dir
transform = transforms.Compose([
    transforms.Resize((224, 224)),    # redimensionando as imagens para ficar de acordo com o modelo
    transforms.ToTensor(),            # Convertendo imagem para tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizando de acordo com os padrões ImageNet
])

class_names = ["NORMAL", "DOENTE"]  # definindo as classes

# Loop para analisar uma quantidade n de imagens aleatórias e ver resultados
n = 10
for i in range(n):
    test_image_path, actual_class = get_random_image_from_folder(test_folder_path)  # Selecionar imagem
    original_image, probabilities = predict_image(model, test_image_path, transform, device)  # Fazer predição
    visualize_predictions(original_image, probabilities, class_names)  # Visualizar resultados

    # Exibir classe real e classe prevista
    predicted_class_idx = probabilities.argmax()
    predicted_class = class_names[predicted_class_idx]
    print(f"Imagem {i+1}:")
    print(f"  Classe real: {actual_class}")
    print(f"  Classe prevista: {predicted_class}")
    print("-" * 30)

In [ ]:
# Função de avaliação do modelo
def evaluate_model(model, dataloader, class_names):
    model.eval() # colocando o modelo no modo de avaliação
    running_corrects = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

            # Obtendo todas as previsões e classes reais
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculando acurácia
    accuracy = running_corrects.double() / len(dataloader.dataset) * 100
    print(f'Test Accuracy: {accuracy:.4f}%')

    # Calculando previsão, recall, and F1-score
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    print(f'Precisão: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

    # Montar matriz de confusão
    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Exibir matriz de confusão
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.matshow(cm, cmap=plt.cm.Blues, alpha=0.6)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(x=j, y=i, s=cm[i, j], va='center', ha='center')

    plt.xlabel('Classes previstas')
    plt.ylabel('Classes verdadeiras')
    plt.title('Matriz de confusão')
    plt.xticks(ticks=[0, 1], labels=class_names)
    plt.yticks(ticks=[0, 1], labels=class_names)
    plt.show()

# Example usage:
class_names = ["NORMAL", "DOENTE"]
evaluate_model(model, dataloaders['test'], class_names)

In [ ]:
# Após o treinamento
model.load_state_dict(best_model_wts)

# Caminho para salvar os pesos
path_to_save = "best_model_weights.pth"

# Salvar os pesos
torch.save(best_model_wts, path_to_save)
print(f"Pesos do melhor modelo salvos em {path_to_save}")


In [ ]:
checkpoint_path = "checkpoint.pth"

# Salvar checkpoint
torch.save({
    'model_state_dict': best_model_wts,
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'epoch': epoch,
    'train_losses': train_losses,
    'val_losses': val_losses
}, checkpoint_path)
print(f"Checkpoint salvo em {checkpoint_path}")
